In [1]:
import sys
sys.path.append("../") # go to parent dir
# imports
import simpy
from loguru import logger
import time
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run

Setting the Cluster

In [2]:
# sim env an data
env = simpy.Environment()
data = simpy.Store(env)
# tier perfs
nvram_bandwidth = {'read':  {'seq': 800, 'rand': 800},
                        'write': {'seq': 400, 'rand': 400}}
ssd_bandwidth = {'read':  {'seq': 200, 'rand': 200},
                    'write': {'seq': 100, 'rand': 100}}
hdd_bandwidth = {'read':  {'seq': 80, 'rand': 80},
                    'write': {'seq': 40, 'rand': 40}}

# registering Tiers
hdd_tier = Tier(env, 'HDD', bandwidth=hdd_bandwidth, capacity=1e12)
ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth,
                        capacity=10e9)
# registering Ephemeral Tier
bb = EphemeralTier(env, name="BB", persistent_tier=hdd_tier,
                        bandwidth=nvram_bandwidth, capacity=10e9)

# Define the cluster with 1 persistent and 1 ephemeral
cluster = Cluster(env, tiers=[hdd_tier], ephemeral_tier=bb)

Behavior for no SBB app

In [3]:
# logger
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [False, False]
app1 = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app1.run(cluster, placement=placement, use_bb=use_bb))
env.run()
fig = display_run(data, cluster, width=800, height=900)
fig.show()
print(f"application duration = {app.get_fitness()}")

NameError: name 'app' is not defined

In [4]:
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [False, True]
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
fig = display_run(data, cluster, width=800, height=900)
fig.show()
print(f"application duration = {app.get_fitness()}")

application duration = 295.0
